In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [2]:
train = pd.read_csv("train.csv", sep='|')
test = pd.read_csv("test.csv", sep='|')

In [3]:
train.head()

,overall,reviewText
0,0,Entertaining enough for those who don't think ...
1,1,I bought it yesterday havent started watching ...
2,1,This movie tells the story of three kids who g...
3,1,You wanna know what its like for a Black perso...
4,1,Warner Archive has finally released an epic fi...


In [4]:
train.overall.value_counts()

1    25000
0    15000
Name: overall, dtype: int64

In [5]:
test.head()

,index,reviewText
0,0,So many reviewers have already given you the p...
1,1,The film opens up with an ideal family. When t...
2,2,Why does everyone like this stupid movie so mu...
3,3,Fair watch. Unlike some shows it got better wi...
4,4,I thought I was buying the DVD of the Season 4...


In [6]:
pipeline_lr = Pipeline([('vectorizer', CountVectorizer()),
                     ('clf_lr', LogisticRegression())])

In [7]:
from sklearn.metrics import f1_score

In [8]:
scores = cross_val_score(pipeline_lr, train.reviewText, train.overall, cv=3, n_jobs=-1, scoring="roc_auc")

In [9]:
scores.mean()

0.9246222941435055

In [10]:
pipeline_lr.fit(train["reviewText"],train["overall"])

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [17]:
pipeline_lr.predict_proba(test["reviewText"])[:,1]

array([  3.53513575e-01,   9.59114280e-01,   4.09530778e-04, ...,
         9.84587913e-01,   1.69911527e-03,   3.33203133e-01])

In [18]:
test["overall"] = pipeline_lr.predict_proba(test["reviewText"])[:,1]

In [19]:
test[["index","overall"]].head()

,index,overall
0,0,0.353514
1,1,0.959114
2,2,0.000410
3,3,0.922766
4,4,0.019038


In [20]:
test[["index","overall"]].to_csv("baseline_submission.csv", index=False)